In [142]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#get local issuer certificate
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [143]:
# Initialize a Chrome WebDriver instance to open chrome
driver = webdriver.Chrome()

In [144]:
# Define the base URL of Facebook (basic mobile version)
facebook_url = 'https://mbasic.facebook.com/'

# Navigate to the base Facebook URL using the Selenium browser
driver.get(facebook_url)

In [145]:
# Click to Accept Facebook Cookies
element = WebDriverWait(driver, 4).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
)
element.click()

In [146]:
# Wait for useremail and password elements to be clickable
useremail_locator = (By.CSS_SELECTOR, "input[name='email']")
password_locator = (By.CSS_SELECTOR, "input[name='pass']")
useremail = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(useremail_locator))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(password_locator))

# Enter usermail and password
useremail.clear()
useremail.send_keys("your_facebook_email")
password.clear()
password.send_keys("your_facebook_password")

# Find and click the login button
login_btn_locator = (By.XPATH, '//*[@id="login_form"]/ul/li[3]/input')
login_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(login_btn_locator))
login_btn.click()

In [147]:
#Skip Facebook Log in with a simple gesture question.
login_btn = driver.find_element(By.XPATH, '//*[@id="root"]/table/tbody/tr/td/div/div[3]/a')
login_btn.click()

#### Recherche des publications

In [157]:
# Search for the input element using WebDriverWait
search = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='query']")))

# Clear the input field and enter the desired hashtag
search.clear()
search.send_keys("#photos")

# Click on the search button
login_btn = driver.find_element(By.XPATH, '//*[@id="header"]/form/table/tbody/tr/td[3]/input')
login_btn.click()

#### Extraction des contenu des publications avec BeautifulSoup

In [158]:
from bs4 import BeautifulSoup
import re

def scrape_links(links):
    # Initialize list to store publications
    publications = []
    for link in links:
        pub = {}

        # Extract href attribute from the link
        href = link.get_attribute('href')

        # Open link in a new tab
        driver.execute_script("window.open('" + href + "', 'new_window')")

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])

        # Wait for the page to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "header")))

        # Get the page source
        page_content = driver.page_source
        
        # Parse page content with BeautifulSoup
        soup = BeautifulSoup(page_content, 'html.parser')

        # Get post text
        paragraphs = soup.find_all('p')
        concatenated_text = ' '.join([p.text for p in paragraphs])
        pub['text'] = concatenated_text.strip()

        # Get image source
        container = soup.find('div', id='objects_container')
        img = container.find('img', src=lambda s: s and s.startswith('https://scontent-cdg4-2.xx.fbcdn.net/')) if container else None
        if img:
            img_url = img["src"]
            pub['image'] = img_url
        else:
            pub['image'] = ''

        ### Get comments
        comments_list = []

        #regular expression
        pattern = r'^composer-'
        
        # Find the div comming before comment div
        #composer_div = soup.find('div', id=lambda x: x and x.startswith('composer-'))
        composer_div = soup.find('div', id=re.compile(pattern))

        if composer_div:
            comment_container = composer_div.find_next_sibling('div')

            comment_divs = comment_container.find_all('div')

            # Extract text from each comment div
            for div in comment_divs:
                if div.text.strip():
                    comments_list.append(div.text.strip())
        
        #add comment to pub
        pub['comments'] = comments_list

        # Append publication to list
        publications.append(pub)

        # Close the current tab
        driver.close()

        # Switch back to the original tab
        driver.switch_to.window(driver.window_handles[0])
    
    return publications


In [159]:
# Find the posts element
posts = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]')

# Find all links with the text "Actualité intégrale"
links = posts.find_elements(By.PARTIAL_LINK_TEXT, "Actualité intégrale")

# Passing the links to scrape
publications = scrape_links(links)

In [160]:
#display collected posts
publications

[{'text': '#Photos Bonjour à tous, nous avons commencé à mettre en ligne quelques photos de cette édition 2024 de la course de la Dentelle',
  'image': '',
  'comments': ['Uca Caudry@followersJ’aime · Réagir · Répondre · Plus · il y a 2 heures',
   'Uca Caudry@followersJ’aime · Réagir · Répondre · Plus · il y a 2 heures',
   '@followers',
   'J’aime · Réagir · Répondre · Plus · il y a 2 heures']},
 {'text': 'Très fier d’avoir participé à cette belle campagne - merci @rtl_france pour votre confiance - Photos de la campagne @agence_1827 #rtl #agence1827 #photos #pub #radio #communication',
  'image': '',
  'comments': ['Laurent MarsickAh… c’est le retour du maestro à RTL ????J’aime · Réagir · Répondre · Plus · Il y a une heure',
   'Laurent MarsickAh… c’est le retour du maestro à RTL ????J’aime · Réagir · Répondre · Plus · Il y a une heure',
   'Ah… c’est le retour du maestro à RTL ????',
   'J’aime · Réagir · Répondre · Plus · Il y a une heure',
   'David RaguetLire la vidéoJ’aime · Réa

#### Stocker les textes et les images dans une base MongoDB.

In [2]:
import pymongo

# Define MongoDB connection parameters
mongodb_uri = "mongodb://localhost:27017/"
database_name = "db_facebook"
collection_name = "posts"

myclient = pymongo.MongoClient(mongodb_uri)
mydb = myclient[database_name]
mycollection = mydb[collection_name]

#Insert posts
x = mycollection.insert_many(publications)

#find post
last_post = mycollection.find_one(sort=[('_id', pymongo.DESCENDING)])

last_post

{'_id': ObjectId('65e85e7ab980911943e7f120'),
 'text': 'Alors qu’est-ce que tu attends??😃🥰profite de la promotion et passe ta commande‼️. #agrandissement #photos à partir de 3000Fcfa 🤩 ☎️ WhatsApp: 6 77 35 83 03 .  📍Situé à l’entrée maeture Biteng..',
 'image': 'https://scontent-cdg4-2.xx.fbcdn.net/v/t45.5328-9/417433886_7157399457706740_1783178574827806033_n.jpg?stp=c0.0.600.600a_dst-jpg_p600x600&_nc_cat=100&ccb=1-7&_nc_sid=b16ce9&_nc_ohc=r4rZNh0sKikAX_ssRBW&_nc_ht=scontent-cdg4-2.xx&oh=00_AfC30vmtZ3w_SCoyQZRvLEvmIkiGPKLe-UnyT8I8MI-7fw&oe=65ED4745',
 'comments': ['Aji univers@followersJ’aime · Réagir · Répondre · Plus · 28 févr., à 22:01',
  'Aji univers@followersJ’aime · Réagir · Répondre · Plus · 28 févr., à 22:01',
  '@followers',
  'J’aime · Réagir · Répondre · Plus · 28 févr., à 22:01',
  'Aji universGIFTO LE RUSSE Officiel J’aime · Réagir · Répondre · Plus · 28 févr., à 22:01',
  'Aji universGIFTO LE RUSSE Officiel J’aime · Réagir · Répondre · Plus · 28 févr., à 22:01',
  'GIFTO